## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df.APPLICATION_TYPE.value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
application_types_to_replace = list(application_counts[application_counts < 500].index)

#  YOUR CODE GOES HERE

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df.CLASSIFICATION.value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df.CLASSIFICATION.value_counts()
classifications_to_replace
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
application_cat = ["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "INCOME_AMT", 
                  "SPECIAL_CONSIDERATIONS"]
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

application_df = application_df.merge(encode_df, left_index=True, right_index=True).drop(columns=application_cat, axis=1)
application_df.head()

c:\users\ptmrt\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL
X = application_df.drop("IS_SUCCESSFUL", axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

X_train_scaled.shape

(25724, 108)

## Compile, Train and Evaluate the Model

In [11]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
nodes_hidden_layer1 = 80
nodes_hidden_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                8720      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 11,181
Trainable params: 11,181
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:

import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}hdf5"

cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 609us/step - loss: 0.5750 - accuracy: 0.7210
Epoch 2/100
804/804 [==============================] - 0s 618us/step - loss: 0.5515 - accuracy: 0.7300
Epoch 3/100
804/804 [==============================] - 0s 602us/step - loss: 0.5488 - accuracy: 0.7314
Epoch 4/100
804/804 [==============================] - 0s 609us/step - loss: 0.5466 - accuracy: 0.7322
Epoch 5/100
766/804 [===========================>..] - ETA: 0s - loss: 0.5449 - accuracy: 0.7312
Epoch 5: saving model to checkpoints\weights.05hdf5
804/804 [==============================] - 1s 637us/step - loss: 0.5452 - accuracy: 0.7316
Epoch 6/100
804/804 [==============================] - 0s 618us/step - loss: 0.5443 - accuracy: 0.7335
Epoch 7/100
804/804 [==============================] - 0s 612us/step - loss: 0.5432 - accuracy: 0.7334
Epoch 8/100
804/804 [==============================] - 0s 606us/step - loss: 0.5422 - accuracy: 0.7331
Epoch 9/100
804/804 [==================

804/804 [==============================] - 1s 1ms/step - loss: 0.5305 - accuracy: 0.7410
Epoch 63/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5302 - accuracy: 0.7416
Epoch 64/100
804/804 [==============================] - 1s 933us/step - loss: 0.5303 - accuracy: 0.7413
Epoch 65/100
766/804 [===========================>..] - ETA: 0s - loss: 0.5303 - accuracy: 0.7420
Epoch 65: saving model to checkpoints\weights.65hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5304 - accuracy: 0.7414
Epoch 66/100
804/804 [==============================] - 1s 992us/step - loss: 0.5304 - accuracy: 0.7413
Epoch 67/100
804/804 [==============================] - 1s 864us/step - loss: 0.5296 - accuracy: 0.7419
Epoch 68/100
804/804 [==============================] - 1s 843us/step - loss: 0.5300 - accuracy: 0.7415
Epoch 69/100
804/804 [==============================] - 1s 956us/step - loss: 0.5300 - accuracy: 0.7414
Epoch 70/100
782/804 [===========================

In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5661 - accuracy: 0.7294 - 331ms/epoch - 1ms/step
Loss: 0.5660807490348816, Accuracy: 0.7294460535049438


In [16]:
number_input_features = len(X_train_scaled[0])
nodes_hidden_layer1 = 80
nodes_hidden_layer2 = 30

nn = tf.keras.models.Sequential()

nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer1, activation="tanh", input_dim=number_input_features))
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer2, activation="tanh"))

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

os.makedirs("checkpoints_optimization_add_layer/", exist_ok=True)
checkpoint_path = "checkpoints_optimization_add_layer/weights.{epoch:02d}hdf5"

cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=0,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback], verbose=1)

Epoch 1/100
804/804 [==============================] - 1s 606us/step - loss: 0.5695 - accuracy: 0.7221
Epoch 2/100
804/804 [==============================] - 1s 659us/step - loss: 0.5530 - accuracy: 0.7296
Epoch 3/100
804/804 [==============================] - 1s 936us/step - loss: 0.5491 - accuracy: 0.7303
Epoch 4/100
804/804 [==============================] - 1s 814us/step - loss: 0.5471 - accuracy: 0.7329
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5448 - accuracy: 0.7324
Epoch 6/100
804/804 [==============================] - 1s 807us/step - loss: 0.5437 - accuracy: 0.7323
Epoch 7/100
804/804 [==============================] - 0s 608us/step - loss: 0.5424 - accuracy: 0.7340
Epoch 8/100
804/804 [==============================] - 0s 604us/step - loss: 0.5418 - accuracy: 0.7340
Epoch 9/100
804/804 [==============================] - 0s 604us/step - loss: 0.5402 - accuracy: 0.7355
Epoch 10/100
804/804 [==============================] - 1s 640us/step - los

804/804 [==============================] - 1s 860us/step - loss: 0.5249 - accuracy: 0.7427
Epoch 80/100
804/804 [==============================] - 1s 919us/step - loss: 0.5246 - accuracy: 0.7434
Epoch 81/100
804/804 [==============================] - 1s 857us/step - loss: 0.5246 - accuracy: 0.7430
Epoch 82/100
804/804 [==============================] - 1s 909us/step - loss: 0.5243 - accuracy: 0.7435
Epoch 83/100
804/804 [==============================] - 1s 950us/step - loss: 0.5247 - accuracy: 0.7430
Epoch 84/100
804/804 [==============================] - 1s 947us/step - loss: 0.5244 - accuracy: 0.7439
Epoch 85/100
804/804 [==============================] - 1s 953us/step - loss: 0.5243 - accuracy: 0.7432
Epoch 86/100
804/804 [==============================] - 1s 990us/step - loss: 0.5244 - accuracy: 0.7432
Epoch 87/100
804/804 [==============================] - 1s 922us/step - loss: 0.5243 - accuracy: 0.7436
Epoch 88/100
804/804 [==============================] - 1s 928us/step - loss:

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5608 - accuracy: 0.7304 - 255ms/epoch - 953us/step
Loss: 0.560765266418457, Accuracy: 0.7303789854049683


In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
nodes_hidden_layer1 = 80
nodes_hidden_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer1, activation="tanh", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer2, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Define the checkpoint path and filenames
os.makedirs("checkpoints_optimization_change_activ/", exist_ok=True)
checkpoint_path = "checkpoints_optimization_change_activ/weights.{epoch:02d}hdf5"
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=0,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback], verbose=1)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 80)                8720      
                                                                 
 dense_10 (Dense)            (None, 30)                2430      
                                                                 
 dense_11 (Dense)            (None, 1)                 31        
                                                                 
Total params: 11,181
Trainable params: 11,181
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
804/804 [==============================] - 1s 628us/step - loss: 0.5716 - accuracy: 0.7212
Epoch 2/100
804/804 [==============================] - 1s 707us/step - loss: 0.5527 - accuracy: 0.7291
Epoch 3/100
804/804 [==============================] - 1s 707us/step - loss: 0.5479 - accuracy: 0.7317
Epoch

804/804 [==============================] - 1s 864us/step - loss: 0.5250 - accuracy: 0.7434
Epoch 72/100
804/804 [==============================] - 1s 851us/step - loss: 0.5252 - accuracy: 0.7430
Epoch 73/100
804/804 [==============================] - 1s 847us/step - loss: 0.5249 - accuracy: 0.7437
Epoch 74/100
804/804 [==============================] - 1s 859us/step - loss: 0.5252 - accuracy: 0.7436
Epoch 75/100
804/804 [==============================] - 1s 879us/step - loss: 0.5250 - accuracy: 0.7427
Epoch 76/100
804/804 [==============================] - 1s 842us/step - loss: 0.5247 - accuracy: 0.7434
Epoch 77/100
804/804 [==============================] - 1s 858us/step - loss: 0.5250 - accuracy: 0.7437
Epoch 78/100
804/804 [==============================] - 1s 852us/step - loss: 0.5246 - accuracy: 0.7433
Epoch 79/100
804/804 [==============================] - 1s 838us/step - loss: 0.5246 - accuracy: 0.7432
Epoch 80/100
804/804 [==============================] - 1s 879us/step - loss:

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5610 - accuracy: 0.7263 - 240ms/epoch - 896us/step
Loss: 0.5610116720199585, Accuracy: 0.7262973785400391


In [20]:
number_input_features = len(X_train_scaled[0])
nodes_hidden_layer1 = 80
nodes_hidden_layer2 = 30
nn = tf.keras.models.Sequential()

nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer1, activation="tanh", input_dim=number_input_features))

nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer2, activation="tanh"))

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 80)                8720      
                                                                 
 dense_13 (Dense)            (None, 30)                2430      
                                                                 
 dense_14 (Dense)            (None, 1)                 31        
                                                                 
Total params: 11,181
Trainable params: 11,181
Non-trainable params: 0
_________________________________________________________________


In [21]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
os.makedirs("checkpoints_optimization_change_activ/", exist_ok=True)
checkpoint_path = "checkpoints_optimization_change_activ/weights.{epoch:02d}hdf5"
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=0,
    save_weights_only=True,
    save_freq='epoch',
    period=5)


fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback], verbose=1)


Epoch 1/100
804/804 [==============================] - 1s 616us/step - loss: 0.5705 - accuracy: 0.7219
Epoch 2/100
804/804 [==============================] - 1s 712us/step - loss: 0.5524 - accuracy: 0.7292
Epoch 3/100
804/804 [==============================] - 1s 986us/step - loss: 0.5480 - accuracy: 0.7313
Epoch 4/100
804/804 [==============================] - 1s 915us/step - loss: 0.5459 - accuracy: 0.7300
Epoch 5/100
804/804 [==============================] - 1s 679us/step - loss: 0.5442 - accuracy: 0.7307
Epoch 6/100
804/804 [==============================] - 1s 724us/step - loss: 0.5431 - accuracy: 0.7326
Epoch 7/100
804/804 [==============================] - 1s 648us/step - loss: 0.5411 - accuracy: 0.7333
Epoch 8/100
804/804 [==============================] - 1s 654us/step - loss: 0.5408 - accuracy: 0.7344
Epoch 9/100
804/804 [==============================] - 1s 662us/step - loss: 0.5397 - accuracy: 0.7343
Epoch 10/100
804/804 [==============================] - 1s 648us/step - l

804/804 [==============================] - 1s 843us/step - loss: 0.5245 - accuracy: 0.7433
Epoch 79/100
804/804 [==============================] - 1s 850us/step - loss: 0.5244 - accuracy: 0.7439
Epoch 80/100
804/804 [==============================] - 1s 881us/step - loss: 0.5243 - accuracy: 0.7438
Epoch 81/100
804/804 [==============================] - 1s 834us/step - loss: 0.5242 - accuracy: 0.7435
Epoch 82/100
804/804 [==============================] - 1s 855us/step - loss: 0.5243 - accuracy: 0.7438
Epoch 83/100
804/804 [==============================] - 1s 840us/step - loss: 0.5245 - accuracy: 0.7432
Epoch 84/100
804/804 [==============================] - 1s 835us/step - loss: 0.5242 - accuracy: 0.7433
Epoch 85/100
804/804 [==============================] - 1s 872us/step - loss: 0.5240 - accuracy: 0.7439
Epoch 86/100
804/804 [==============================] - 1s 853us/step - loss: 0.5242 - accuracy: 0.7439
Epoch 87/100
804/804 [==============================] - 1s 844us/step - loss:

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5617 - accuracy: 0.7293 - 236ms/epoch - 881us/step
Loss: 0.5617451667785645, Accuracy: 0.7293294668197632


In [23]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")